In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  LSTM


2024-02-02 23:54:30.820744: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import yfinance as yf

In [3]:
period = '3y'

tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'TSLA', 'NVDA', 'PYPL', 'ADBE',
           'BTC-USD', 'ETH-USD', 'XRP-USD', 'LTC-USD', 'BCH-USD', 'BNB-USD', 'LINK-USD', 'ADA-USD', 'XLM-USD', 'SOL-USD', 'TRX-USD']

In [4]:
def indir_ve_df_olustur(tickers, period):
    # Her bir hisse senedi için boş bir sözlük oluşturun
    pariteler = {}

    # Her bir hisse senedi için döngü oluşturun ve verileri indirin
    for ticker in tickers:
        try:
            # Hisse senedi verilerini indirin
            veri = yf.download(ticker, period=period)
            
            # Veriyi sözlüğe ekleyin
            pariteler[ticker] = veri
        except Exception as e:
            print(f"{ticker} için veri indirilirken bir hata oluştu: {str(e)}")

    # Her bir hisse senedi için ayrı bir veri çerçevesi oluşturun
    df_listesi = [veri for veri in pariteler.values()]

    return df_listesi

In [5]:
#pariteler adında klasör oluştur :

import os 

if not os.path.exists('pariteler'):
    os.makedirs('pariteler')



In [6]:
datasets = indir_ve_df_olustur(tickers, period)

#pariteler içine kaydet 

for i in range(len(datasets)):
    datasets[i].to_csv(f'pariteler/{tickers[i]}.csv')
    #print(f'{tickers[i]} verisi csv olarak kaydedildi')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [7]:
window = 60

def veri_hazirla(df, window):
    # Veri çerçevesinin kopyasını oluşturun
    df = df.copy()
    
    # Veri çerçevesine yeni sütunlar ekleyin
    for i in range(1, window + 1):
        df[f'Önceki_{i}_Kapanış'] = df['Close'].shift(i)
        df[f'Önceki_{i}_Hacim'] = df['Volume'].shift(i)
        df[f'Önceki_{i}_Açılış'] = df['Open'].shift(i)
        df[f'Önceki_{i}_Yüksek'] = df['High'].shift(i)
        df[f'Önceki_{i}_Düşük'] = df['Low'].shift(i)
        
    # NaN değerleri bırakın
    df.dropna(inplace=True)
    
    return df

# Her bir veri çerçevesi için döngü oluşturun ve verileri hazırlayın
data_windowed = [veri_hazirla(df, window) for df in datasets]


if not os.path.exists('windowed'):
    os.makedirs('windowed')

# Her bir veri çerçevesi için parite ismiyle birlikte windowed klasörüne kaydedin 
for i in range(len(data_windowed)):
    data_windowed[i].to_csv(f'windowed/{tickers[i]}_windowed.csv')
    print(f'{tickers[i]} verisi windowed olarak kaydedildi')



/var/folders/w9/3z_zbj956tj44glgzg44rgbr0000gn/T/ipykernel_45007/2113812001.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Önceki_{i}_Yüksek'] = df['High'].shift(i)
/var/folders/w9/3z_zbj956tj44glgzg44rgbr0000gn/T/ipykernel_45007/2113812001.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Önceki_{i}_Düşük'] = df['Low'].shift(i)
/var/folders/w9/3z_zbj956tj44glgzg44rgbr0000gn/T/ipykernel_45007/2113812001.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

AAPL verisi windowed olarak kaydedildi
MSFT verisi windowed olarak kaydedildi
AMZN verisi windowed olarak kaydedildi
GOOG verisi windowed olarak kaydedildi
GOOGL verisi windowed olarak kaydedildi
TSLA verisi windowed olarak kaydedildi
NVDA verisi windowed olarak kaydedildi
PYPL verisi windowed olarak kaydedildi
ADBE verisi windowed olarak kaydedildi
BTC-USD verisi windowed olarak kaydedildi
ETH-USD verisi windowed olarak kaydedildi
XRP-USD verisi windowed olarak kaydedildi
LTC-USD verisi windowed olarak kaydedildi
BCH-USD verisi windowed olarak kaydedildi
BNB-USD verisi windowed olarak kaydedildi
LINK-USD verisi windowed olarak kaydedildi
ADA-USD verisi windowed olarak kaydedildi
XLM-USD verisi windowed olarak kaydedildi
SOL-USD verisi windowed olarak kaydedildi
TRX-USD verisi windowed olarak kaydedildi


In [10]:
data_windowed

[                  Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2021-04-29  136.470001  137.070007  132.449997  133.479996  131.359543   
 2021-04-30  131.779999  133.559998  131.070007  131.460007  129.371613   
 2021-05-03  132.039993  134.070007  131.830002  132.539993  130.434448   
 2021-05-04  131.190002  131.490005  126.699997  127.849998  125.818954   
 2021-05-05  129.199997  130.449997  127.970001  128.100006  126.064987   
 ...                ...         ...         ...         ...         ...   
 2024-01-29  192.009995  192.199997  189.580002  191.729996  191.729996   
 2024-01-30  190.940002  191.800003  187.470001  188.039993  188.039993   
 2024-01-31  187.039993  187.100006  184.350006  184.399994  184.399994   
 2024-02-01  183.990005  186.949997  183.820007  186.860001  186.860001   
 2024-02-02  179.860001  187.320007  179.250000  186.769897  186.769897   
 
                Volume 

In [9]:
#her bir veri çerçevesi için X  = features y = labels olan veriyapısını kur

X = []
y = []
for data_frame in data:
    X.append(data_frame.features) 
    y.append(data_frame.label)









